# 姿勢演算　クォータニオン操作

### 注意<br>
準備としてnumpy-quaternionをインストールしておく<br>
!pip install numpy-quaternion

参考サイト：https://qiita.com/momomo_rimoto/items/3a245736c5fd90fe8270

In [1]:
import sys
sys.path.append('../')

In [10]:
import numpy as np
import quaternion

In [11]:
class attitude_conv_rev01:
    #準備としてnumpy-quaternionをインストールしておくこと。
    def __init__(self):
        self.frd = np.array([0,0,0])
        self.ned = np.array([0,0,0])
        self.local = np.array([0,0,0])
        self.focal = np.array([0,0,0])
        
    def ned2frd(self, north,east,below,roll,pitch,yaw):
        #入力：　北、東、南 ：単位m
        #入力：ロール、ピッチ、ヨ― ：単位°
        #出力： 前、右、下 単位m
        #オイラーシーケンス：ヨー→ピッチ→ロール
        
        self.ned = np.array([north,east,below])
        q1=quaternion.from_rotation_vector([0,0,np.radians(yaw)])
        q2=quaternion.from_rotation_vector([0,np.radians(pitch),0])
        q3=quaternion.from_rotation_vector([np.radians(roll),0,0])

        quat=q3.inverse()*q2.inverse()*q1.inverse()
        rot = quaternion.as_rotation_matrix(quat)
        self.frd = np.matmul(rot, self.ned)
        
        self.fwd,self.rh,self.dwn = self.frd
        
        return self.fwd,self.rh,self.dwn
    
    
    def frd2ned(self, fwd,rh,dwn,roll,pitch,yaw):
        #入力：　前、右、下 単位m
        #入力：ロール、ピッチ、ヨ― 単位°
        #出力： 北、東、南 ：単位m
        #オイラーシーケンス：ヨー→ピッチ→ロール
        
        self.frd = np.array([fwd,rh,dwn])
               
        q1=quaternion.from_rotation_vector([0,0,np.radians(yaw)])
        q2=quaternion.from_rotation_vector([0,np.radians(pitch),0])
        q3=quaternion.from_rotation_vector([np.radians(roll),0,0])
        
        quat=q1*q2*q3

        rot = quaternion.as_rotation_matrix(quat)
        self.ned = np.matmul(rot, self.frd)
        
        self.north,self.east,self.below = self.ned
        
        return self.north,self.east,self.below
    
    def local2ned(self, x_loc, y_loc, z_loc, ido, keido):
        #入力：　局所X、局所Y、局所Z 単位m
        #入力：緯度、経度　単位°
        #出力： 北、東、南 ：単位m
        #オイラーシーケンス：経度→緯度
        
        self.xyz=np.array([x_loc,y_loc,z_loc])
        
        q1=quaternion.from_rotation_vector([0,np.radians(-90),0])
        q2=quaternion.from_rotation_vector([np.radians(keido),0,0])
        q3=quaternion.from_rotation_vector([0,np.radians(-ido),0])
        
        quat = q3.inverse()*q2.inverse()*q1.inverse()
        
        rot = quaternion.as_rotation_matrix(quat)
        
        self.ned = np.matmul(rot, self.xyz)
        self.north,self.east,self.below = self.ned
        
        return self.north,self.east,self.below
    
    def ned2local(self, north, east, below, ido, keido):
        #入力：北、東、南 ：単位m　
        #入力：緯度、経度　単位°
        #出力： 局所X、局所Y、局所Z 単位m
        #オイラーシーケンス：経度→緯度
        
        self.ned = np.array([north,east,below])

        q1=quaternion.from_rotation_vector([0,np.radians(-90),0])
        q2=quaternion.from_rotation_vector([np.radians(keido),0,0])
        q3=quaternion.from_rotation_vector([0,np.radians(-ido),0])
        
        quat=q1*q2*q3
        
        rot = quaternion.as_rotation_matrix(quat)
        
        self.local = np.matmul(rot, self.ned)
        self.x_loc, self.y_loc, self.z_loc = self.local
        
        return self.x_loc, self.y_loc, self.z_loc
    
    def frd2focal(self, fwd, rh, dwn, spin, el, az):
        #入力：前、右、下 ：単位m　
        #入力：アジマス、エレベーション、スピン　：単位°
        #出力： 水平右、垂直下、奥行前 ：単位m
        #オイラーシーケンス：アジマス→エレベーション→スピン
        self.frd = np.array([fwd,rh,dwn])
        
        q1=quaternion.from_rotation_vector([0,0,np.radians(az)])
        q2=quaternion.from_rotation_vector([0,np.radians(el),0])
        q3=quaternion.from_rotation_vector([np.radians(spin),0,0])
        q4=quaternion.from_rotation_vector([0,0,np.radians(90)])
        q5=quaternion.from_rotation_vector([np.radians(90),0,0])

        quat=q5.inverse()*q4.inverse()*q3.inverse()*q2.inverse()*q1.inverse()
        rot = quaternion.as_rotation_matrix(quat)
        self.focal = np.matmul(rot, self.frd)
        
        self.focal_hor,self.focal_ver,self.focal_dep = self.focal
        
        return self.focal_hor,self.focal_ver,self.focal_dep
    
    def focal2frd(self, focal_hor, focal_ver, focal_dep, spin, el, az):
        #入力： 水平右、垂直下、奥行前 ：単位m
        #入力：アジマス、エレベーション、スピン　：単位°
        #出力：前、右、下 ：単位m　
        #オイラーシーケンス：アジマス→エレベーション→スピン
        
        self.focal = np.array([focal_hor,focal_ver,focal_dep])
        
        q1=quaternion.from_rotation_vector([0,0,np.radians(az)])
        q2=quaternion.from_rotation_vector([0,np.radians(el),0])
        q3=quaternion.from_rotation_vector([np.radians(spin),0,0])
        q4=quaternion.from_rotation_vector([0,0,np.radians(90)])
        q5=quaternion.from_rotation_vector([np.radians(90),0,0])

        quat=q1*q2*q3*q4*q5
        rot = quaternion.as_rotation_matrix(quat)
        self.frd = np.matmul(rot, self.focal)
        
        self.fwd,self.rh,self.dwn = self.frd
        
        return self.fwd,self.rh,self.dwn
    
    
    
        

### 機体位置を原点に、NED座標で表された位置(真北、東、真下）を、機体固定座標(前方、右方、下方）で表すサンプル<br>角度単位は°

In [4]:
roll = 0; pitch = -45; yaw =90
north =0; east=1; below=0
fwd, rh, dwn = attitude_conv_rev01().ned2frd(north,east,below,roll,pitch,yaw)
print(fwd, rh, dwn)

0.7071067811865474 3.3306690738754696e-16 -0.7071067811865474


### 機体位置を原点に、機体固定座標で表された位置（前方、右方、下方）を、NED座標（真北、東、真下）で表すサンプル<br>（角度単位は°）

In [5]:
north, east, below = attitude_conv_rev01().frd2ned(fwd,rh,dwn,roll,pitch,yaw)
print(north, east, below)

-3.3306690738754696e-16 0.9999999999999998 5.551115123125783e-17


### 機体位置を原点に機体局所座標で表された位置（局所X、局所Y、局所Z）を、NED座標（真北、東、真下）で表すサンプル。<br>（角度単位は°）（局所X方向は、地球中心から北緯0°東経0°の地点を見た方向と平行）

In [6]:
x_loc=0.2; y_loc=0.4; z_loc=0.6;
ido=30;keido=90
north, east ,below = attitude_conv_rev01().local2ned(x_loc,y_loc,z_loc,ido,keido)
print(north, east, below)

0.31961524227066307 -0.19999999999999987 -0.6464101615137753


### 機体位置を原点に、NED座標で表された位置（真北、東、真下）を、機体局所座標で表された位置（局所X、局所Y、局所Z）で表すサンプル。<br>（角度単位は°）（局所X方向は、地球中心から北緯０°東経０°の地点を見た方向と平行）

In [7]:
x_loc, y_loc, z_loc = attitude_conv_rev01().ned2local(north, east, below, ido, keido)
print(x_loc, y_loc, z_loc)

0.19999999999999982 0.40000000000000013 0.6000000000000005


### 機体位置を原点に、機体固定座標で表された位置（前方、右方、下方）を、機体局所座標で表されたセンサ固定座標（水平、垂直、奥行）で表すサンプル。<br>（角度単位は°）（水平は右が正、垂直は下が正）

In [8]:
fwd=0.2; rh=0.4; dwn=0.6;
spin=0; el=90; az=0;
focal_hor, focal_ver, focal_dep = attitude_conv_rev01().frd2focal(fwd, rh, dwn, spin, el, az)
print(focal_hor, focal_ver, focal_dep)

0.39999999999999997 0.2000000000000001 -0.5999999999999994


### 機体位置を原点に、センサ固定座標で表された位置（水平、垂直、奥行）を、機体局所座標（前方、右下、下方）で表すサンプル。<br>（角度単位は°）（水平は右が正、垂直は下が正）

In [9]:
fwd, rh, dwn = attitude_conv_rev01().focal2frd(focal_hor, focal_ver, focal_dep, spin, el, az)
print(fwd, rh, dwn)

0.2 0.4 0.6
